# Tutorial 2 - The Queens & The Art of Branching (NOTES Persos: precise the vars and value orderings). The ones thgat make the observations waited for


**NOTE : You must fully finish Tutorial i-1 before starting Tutorial i (i>1)**

Constraint programming is used to solve highly combinatorial and complex problems. 
In order to master CP, you need to get used to the philosophy behind this approach as well as the different 
techniques used along with it. We will guide you in this process step by step in the upcoming tutorials. We will be using "toy" puzzles/problems only for the purpose of learning different faces of CP. In real life problems, things get messed up easily and require decision and policy makes to agree opon the problem at hand.. (Trust us, we've been there..) 

In this tutorial, we use the N-Queens problem, one of oldest and classical problems solved efficiently by CP, as a case study. 

## The N-Queens problem

You are given an N-by-N chessboard, and your goal is to place N chess queens on it so that no two queens threaten each other:

<div class="row" style="margin-top: 10px">
    <div class="col-md-5">
        <img src="display/images/empty-chessboard.png" style="margin-right: 0; width: 160px;" />
    </div>
    <div class="col-md-2" style="display: table">
        <i class="fa fa-arrow-right" style="display: table-cell; font-size: 50px; 
        margin: auto; text-align: center; vertical-align: middle; height: 150px"></i>
    </div>
    <div class="col-md-5">
        <img src="display/images/nqueens8-chessboard.png" style="margin-left: 0; width: 160px;" />
    </div>
</div>

Formally, a solution to the N-queens problem requires that no two queens share the same row, column or diagnoal.

### First model without global constraints

**Exercice**: Create a function `decomposition_model(N)` that models the problem using only binary inequality constrants (no global constraint) and returns an instance of `CpoModel` for the n-queens problem with `N` queens.

In [1]:
from config import setup
setup()

In [17]:
from docplex.cp.model import CpoModel

def decomposition_model(N):
    mdl = CpoModel(name='nqueens')
    q = mdl.integer_var_list(N, 0, N - 1, 'q')
    for i in range(N):
        for j in range(i + 1, N):
            mdl.add(q[i] != q[j])
            mdl.add(q[j] - q[i] != j - i)
            mdl.add(q[i] - q[j] != j - i)
    return mdl, q

Test your function by solving the n-queens problem for small values of $N$ ($< 20$).

*Note: Use the `display.n_queens` function in order to display a solution for the n-queens problem. This function can take a list of `int` corresponding to the column of the queens in order to display them.*

In [18]:
from display import n_queens as display_queens

mdl, queens = decomposition_model(27)
 
sol = mdl.solve()

display_queens([sol[q] for q in queens])

 ! ----------------------------------------------------------------------------
 ! Satisfiability problem - 27 variables, 1053 constraints
 ! Workers              = 1
 ! Presolve             = Off
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 128.4 (before), 128.4 (after)
 !  . Memory usage      : 468.4 kB (before), 468.4 kB (after)
 ! Using sequential search.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed            Branch decision
 *                    434  0.00s                  4  = q_24
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 434
 ! Number of fails        : 199
 ! Total memory usage     : 1.7 MB (1.7 MB CP Optimizer + 0.0 MB Concert)
 ! Time spent in solve    : 0.00s (0.00s engi

<Figure size 1350x1350 with 1 Axes>

**Question**: How many solutions are there for $N = 3,~\ldots,~10$? 

**Note:** To answer this question, you must force the solver to use a depth first strategy: ... start_search(SearchType='DepthFirst')


In [19]:
for n in range(3, 10 + 1):
    mdl, _ = decomposition_model(n)
    sols = list(mdl.start_search(SearchType='DepthFirst', LogVerbosity='Quiet'))
    print('N={}, {} solution(s) found.'.format(n, len(sols)))

N=3, 0 solution(s) found.
N=4, 2 solution(s) found.
N=5, 10 solution(s) found.
N=6, 4 solution(s) found.
N=7, 40 solution(s) found.
N=8, 92 solution(s) found.
N=9, 352 solution(s) found.
N=10, 724 solution(s) found.


### Second model with global constraints

Create a function `global_constraint_model(N)` that models and returns an instance of `CpoModel` for the n-queens problem with `N` queens, using **only** and exaclty 3 global constraints.

In [20]:
from docplex.cp.model import CpoModel

def global_constraint_model(N):
    mdl = CpoModel(name='nqueens')
    q = mdl.integer_var_list(N, 0, N - 1, 'q')
    mdl.add(mdl.all_diff(q))
    mdl.add(mdl.all_diff([q[i] + i for i in range(N)]))
    mdl.add(mdl.all_diff([q[i] - i for i in range(N)]))
    return mdl, q

Test your function by solving the n-queens problem for small values of  N  ( <20 ).

In [12]:
#Similar as above

How many solutions are there for  $N=3, .., 10$ ? It must be the same number as the previous model! 

In [13]:
#Similar as above

### Comparison of the two models

We will compare the two models properly. Consider the run(model,params) function that we used in the first tutorial. We will used it here.

To comprate the two models, we will fix the parameters to be : 
    
    TimeLimit=10
    
And we keep all the other parameters by default    

In [21]:
## Here they need to plot . the results.. 

from docplex.cp.parameters import CpoParameters

def run(model, params): 
#    sol = model.solve(TimeLimit= params.TimeLimit , LogPeriod = params.LogPeriod )
    sol = model.solve(TimeLimit= params.TimeLimit , LogVerbosity="Quiet" )
    #print(sol.get_solver_log())
    return sol.get_solver_infos()['NumberOfChoicePoints'], sol.get_solver_infos()['TotalTime'] , sol.get_solve_status() 

parameters = CpoParameters(TimeLimit=10, SearchType='DepthFirst')

mdl , x = global_constraint_model(2000)
run(mdl,parameters)
sol = mdl.solve( )
sol.print_solution()


 ! ----------------------------------------------------------------------------
 ! Satisfiability problem - 2000 variables, 3 constraints
 ! Workers              = 1
 ! Presolve             = Off
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 21931.6 (before), 21931.6 (after)
 !  . Memory usage      : 1.6 MB (before), 1.6 MB (after)
 ! Using sequential search.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed            Branch decision
                     1000       1001           1991  = q_1034
                     2000          7        F  1727 != q_1955
 *                   2063  0.29s                381  = q_1911
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! ----------------------------------------------------------------------------
 ! Number of branches     : 2063
 ! Number of fails       

What do you observe? Why? 

### Branching strategies

Before you start this part, please ask one of your supervisors to check on your work! This is mandatory for the rest! 


Exciting things are about to start. A CP solver is, at the end of the day, a backtracking solver. At each node, it applies filtering (called also propagation or pruning), then make a decision about the next node to explore. This decision is a pure heuristic choice! That is, it could be a wrong decision. It's only based on intuition. 

In CP, a decision is, most of the times, of the type: Choose an unassigned variable $x$, choose a value $v$ from the current domain of $x$, and assign $v$ to $x$. These steps require a variable heuristic and a value heuristic. This is what we call branching strategy. 

Branching strategies can be generic (strategies that can be used for any problem), or specific (designed for the problem at hand). In CPOptimizer, there are a number of genereic strategies offered. This concerns both variable and value heuristics. 

For example, if $L$ is the list of decision variables, then by declaring a search_phase as follows: 
```
SearchPhase= model.search_phase(L, 
                                varchooser=model.select_smallest(model.domain_size()),
                                valuechooser=model.select_random_value())

model.add_search_phase(SearchPhase)
```
The variable heuristic here is the one that picks the variable $x$ with the smallest domain size, and assigns a random value from its domain to it. 


Read about the different search strategies here : 
http://ibmdecisionoptimization.github.io/docplex-doc/cp/docplex.cp.modeler.py.html#search-phases


We want to evaluate different strategies: 
- For variable evaluators, we will use 
 - domain_size() 
 - var_impact()
 - var_local_impact() 
 - var_index()
 
- For value evaluators, we will use 
 - value_impact()
 - value_index()
 
As for the selectors we will use : 
- select_smallest() 
- select_largest ()

How many strategies are we going to evaluate? 

In addition to the above strategies, we will use also a pure random selection for variables and values. How can we use such a branching heuristic? 

Using the global constraints model, run the different strategies (all of them! + pure random) for different values of $N$. Make sure you choose a good value of $N$ that can assess your claims of evaluation. Feel free to use any reasnable time limit. 

**Requirement for this exercice : You must use the parameter**

```
SearchType='DepthFirst'
```
**We will investigate deeply this choice in the next tutorial. But at this stage you should trust us and include it in the solver's parameters.**  


Present the results (rutime + number of nodes) via a table first then using plots. 

In [22]:
mdl, queens = global_constraint_model(3000)

sf = mdl.search_phase(queens, 
                                      varchooser=mdl.select_smallest(mdl.domain_size()),
                                      valuechooser=mdl.select_random_value())
mdl.add_search_phase(sf)
sol = mdl.solve(TimeLimit=30)
print (sol)

 ! ----------------------------------------------------------------------------
 ! Satisfiability problem - 3000 variables, 3 constraints, 1 phase
 ! TimeLimit            = 30
 ! Workers              = 1
 ! Presolve             = Off
 ! Initial process time : 0.00s (0.00s extraction + 0.00s propagation)
 !  . Log search space  : 34652.2 (before), 34652.2 (after)
 !  . Memory usage      : 2.2 MB (before), 2.2 MB (after)
 ! Using sequential search.
 ! ----------------------------------------------------------------------------
 !               Branches  Non-fixed            Branch decision
                     1000       2001           2700  = q_928
                     2000       1001           2204  = q_2015
                     3000          7            223  = q_2892
 *                   3019  0.69s               2274  = q_2947
 ! ----------------------------------------------------------------------------
 ! Search completed, 1 solution found.
 ! ------------------------------------

Is this what you expect? Is the choice of the branching strategy important? Justify

Well I hope the answer is yes

What is more important, the variable ordering or the value ordering choice? Justify

What is the best variable ordering choice? Justify

--

What is the best value ordering choice? Justify

--

What is a good branching overall? Any thoughts why this is the case? Justify

--

Did you observe an opposite behaviour of heuristics betwen the runtime and the number of nodes? 
For instance, is there a strategy that is faster than others to solve the problem but requires a larger number of nodes? 
And conversely, is there a heuristic that is slow to solve the problem than others but uses less nodes? 
If you observe this, why is this happening? 

What did you learn today? 